In [ ]:
### Mount google drive if available
try:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/term_paper/'
    in_colab = True
except:
    drive_path = ''
    in_colab = False

In [ ]:
### Import the utils files into the folder, as well as the files main.py and neural_rendering.py
# Later on, just git clone the repo and run repo_name/main.py
!mkdir utils
!mkdir lightweight_gan

## Humbi human body reconstruction

In [ ]:
### Install all dependecies

# pytorch3d
import os
import sys
import torch

need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True

if need_pytorch3d:
    if torch.__version__.startswith("1.10.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'


# smplx
need_smplx=False
try:
    import smplx
except ModuleNotFoundError:
    need_smplx=True

if need_smplx:
    !pip install smplx
    !git clone https://github.com/vchoutas/smplx
    %cd smplx
    !python setup.py install
    %cd ..


# detectron2
need_detectron=False
try:
    import detectron2
except ModuleNotFoundError:
    need_detectron=True   

if need_detectron:
    !git clone https://github.com/facebookresearch/detectron2.git detectron2_repo
    !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'


# cleanup
!rm -rf 1.10.0.tar.gz cub-1.10.0/

In [ ]:
### CAREFUL: Using the --subdivision flag disables saving displacement map as a texture
if in_colab:
    %run main.py --subjects '[1, 70, 122]' --gdrive {drive_path} --saveobj --smoothing
else:
    %run main.py --subjects '[1, 70, 122]' --saveobj --smoothing

In [ ]:
### Reconstructed displacement texture after inverse normalization 
import numpy as np
import matplotlib.pyplot as plt
from utils.normalize_disps import denormalize_disp

subject = 1
img_path = drive_path + 'humbi_output/humbi_smplx_geom/disp_texture_%d.png' % subject
obj_path = drive_path + 'smplx/smplx_uv.obj'

normalization = np.load(drive_path + 'humbi_output/humbi_smplx_npz/normalization.npz')
global_min = normalization['global_min'].item()
global_max = normalization['global_max'].item()

image, geometry = denormalize_disp(img_path, global_min, global_max, obj_path)

plt.figure(figsize=(10, 10))
plt.imshow(image + 0.5)
plt.axis("off")

## Generative human body modelling

In [ ]:
### Lightweight GAN dependencies (see in setup.py)
!pip install fire
!pip install retry
!pip install kornia==0.5.4
!pip install einops
!pip install adabelief_pytorch

In [ ]:
training_imgs = drive_path + 'humbi_output/humbi_smplx_rgb/'
training_disps = drive_path + 'humbi_output/humbi_smplx_geom/'

models_dir = drive_path  + 'humbi_output/humbi_gan/model/'
results_dir = drive_path + 'humbi_output/humbi_gan/results/'

run_name_rgb = 'humbi_gan_rgb'
run_name_rgbxyz = 'humbi_gan_rgbxyz'

In [ ]:
### RGB only training
%run lightweight_gan/cli.py --data {training_imgs} \
                            --name {run_name_rgb} \
                            --models-dir {models_dir} \
                            --results-dir {results_dir} \
                            --save-every 1000 \
                            --batch-size 8 \
                            --gradient-accumulate-every 4 \
                            --num-train-steps 200000 \
                            --image-size 1024

In [ ]:
### RGB + disps training
%run lightweight_gan/cli.py --data {training_imgs} \
                            --disp_data {training_disps} \
                            --rgbxyz \
                            --name {run_name_rgbxyz} \
                            --models-dir {models_dir} \
                            --results-dir {results_dir} \
                            --save-every 1000 \
                            --batch-size 8 \
                            --gradient-accumulate-every 4 \
                            --num-train-steps 200000 \
                            --image-size 1024

In [ ]:
### Generating from RGB
%run lightweight_gan/cli.py --generate \
                            --name {run_name_rgb} \
                            --models-dir {models_dir} \
                            --results-dir {results_dir} \
                            --generate-types default \
                            --num-image-tiles 16 \
                            # --load-from {checkpoint num}

In [ ]:
### Generating from RGB + disps
%run lightweight_gan/cli.py --rgbxyz \
                            --generate \
                            --name {run_name_rgbxyz} \
                            --models-dir {models_dir} \
                            --results-dir {results_dir} \
                            --generate-types default \
                            --num-image-tiles 16 \
                            # --load-from {checkpoint num}

In [ ]:
### Show progress
%run lightweight_gan/cli.py --show-progress \
                            --name {run_name_rgb} \
                            --models-dir {models_dir} \
                            --results-dir {results_dir} \
                            --generate-types default \
                            --num-image-tiles 1

'''
### Progress to video
progress_imgs = drive_path + results_dir + run_name_rgb + '-progress/*.jpg'
progress_video = drive_path + results_dir + run_name_rgb + '.mp4'

!ffmpeg -framerate 10 -pattern_type glob -i '{progress_imgs}' {progress_video}
print('video created and saved under', progress_video)
'''